# 画像の前処理
※jupyter lab で実行

In [1]:
# ライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import pandas as pd
import cv2
import os
import glob
import random
import gc

from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
# 画像名を変更 
path0 = "./chest_xray"
path_li1 = ["test", "train"]
path_li2 = ["NORMAL", "PNEUMONIA"]
'''
for path1 in path_li1:
    for path2 in path_li2:
        files = glob.glob(path0 + "/" + path1 + "/" + path2 + "/*")
        
        for i, f in enumerate(files):
            ftitle, fext = os.path.splitext(f)
            
            os.rename(f, path0 + "/" + path1 + "/" + path2 + "/" + \
                      path1 + "_" + path2 + "_" + "{0:04d}".format(i) + fext)
'''
print("時間がかかるためコメントアウト")

時間がかかるためコメントアウト


In [3]:
# 画像数を確認
dir_train = path0 + "/train"
dir_test = path0 + "/test"
img_train_li = glob.glob(dir_train + "/*/*.jpeg")
img_train_n_li = glob.glob(dir_train + "/NORMAL/*.jpeg")
img_train_p_li = glob.glob(dir_train + "/PNEUMONIA/*.jpeg")
img_test_li = glob.glob(dir_test + "/*/*.jpeg")
img_test_n_li = glob.glob(dir_test + "/NORMAL/*.jpeg")
img_test_p_li = glob.glob(dir_test + "/PNEUMONIA/*.jpeg")
print(f'# of train images     : {len(img_train_li)}')
print(f'# of normal images    : {len(img_train_n_li)}')
print(f'# of pneumonia images : {len(img_train_p_li)}')
print(f'# of test images      : {len(img_test_li)}')
print(f'# of normal images    : {len(img_test_n_li)}')
print(f'# of pneumonia images : {len(img_test_p_li)}')

# of train images     : 5232
# of normal images    : 1349
# of pneumonia images : 3883
# of test images      : 624
# of normal images    : 234
# of pneumonia images : 390


In [4]:
def scale_to_width(img, width):
    
    # 幅が指定した値になるように、アスペクト比を固定して、リサイズする
    h, w, _ = img.shape
    height = round(h * (width / w))
    dst = cv2.resize(img, dsize=(width, height))

    return dst

## 訓練データの前処理

In [5]:
"""
for i, name in enumerate(img_train_li):
    
    # 画像の読み込み
    img_name = name
    img = cv2.imread(img_name)
        
    # リサイズ
    img = scale_to_width(img, 224)

    # 左右をトリミング
    num_trim = 20
    img = img[:, num_trim:-num_trim]
    
    # リサイズ
    h, w, _ = img.shape
    img = cv2.resize(img, dsize=(224, 224))
    
            
    # 画像の保存
    img_name_224 = img_name[:12] + "_2_224" + img_name[12:]
    cv2.imwrite(img_name_224, img)
"""
print("保存済みのためコメントアウト")

保存済みのためコメントアウト


In [7]:
# 224サイズ用の画像名リスト作成
img_train_224_li = []
for i, name in enumerate(img_train_li):
    
    img_name = name
    img_name_224 = img_name[:12] + "_2_224" + img_name[12:]
    img_train_224_li.append(img_name_224)

In [8]:
# データの形状
h, w, c = 224, 224, 3

# データ領域の大きさ
size = 1

# ndarrayのデータを保管する領域の確保
X_len = len(img_train_224_li)
X_train = np.empty((X_len*size, h*w*c), dtype=np.uint8)

for i, name in enumerate(img_train_224_li):

    # 元の画像をndarrayとして読み込んで訓練データに追加
    img_name = name
    img = cv2.imread(img_name)
    img = np.asarray(img, dtype=np.uint8)
    img_f = img.flatten()
    X_train[i] = img_f

X_train = X_train.reshape(X_len*size, h, w, c)

In [9]:
# 目的変数の作成
Y_train = [[1, 0]] * len(img_train_n_li) + [[0, 1]] * len(img_train_p_li)
Y_train = Y_train * size
Y_train = np.asarray(Y_train)

In [10]:
# # データの保存
# np.save('./X', X_train)
# np.save('./Y', Y_train)

In [12]:
# 訓練データ、検証データの分割
num_n = len(img_train_n_li)
num_p = len(img_train_p_li)

# NormalとPneumoniaに分けて格納
X_n = X_train[:num_n]
X_p = X_train[num_n:]
Y_n = Y_train[:num_n]
Y_p = Y_train[num_n:X_len]
Y_n = np.array(Y_n)
Y_p = np.array(Y_p)

del X_train, Y_train
gc.collect()# メモリ開放

# NormalとPneumoniaをそれぞれ訓練、検証データに分割
X_train_n, X_valid_n, Y_train_n, Y_valid_n = train_test_split(X_n, Y_n, test_size=0.2, random_state=82)
X_train_p, X_valid_p, Y_train_p, Y_valid_p = train_test_split(X_p, Y_p, test_size=0.2, random_state=82)

del X_n, Y_n, X_p, Y_p
gc.collect() # メモリ開放

# 訓練データの構造
print(f"X_train (Normal   ): {X_train_n.shape}")
print(f"X_train (Pneumonia): {X_train_p.shape}")
print(f"X_valid (Normal   ): {X_valid_n.shape}")
print(f"X_valid (Pneumonia): {X_valid_p.shape}")

# データの結合
X_train = np.concatenate([X_train_n, X_train_p])
X_valid = np.concatenate([X_valid_n, X_valid_p])
Y_train = np.concatenate([Y_train_n, Y_train_p])
Y_valid = np.concatenate([Y_valid_n, Y_valid_p])

In [13]:
# 訓練データと検証データの構造
print(f"X_train: {X_train.shape}")
print(f"Y_train: {Y_train.shape}")
print(f"X_valid: {X_valid.shape}")
print(f"Y_valid: {Y_valid.shape}")

X_train: (4185, 224, 224, 3)
Y_train: (4185, 2)
X_valid: (1047, 224, 224, 3)
Y_valid: (1047, 2)


In [14]:
# データの保存
np.save('./X_train2', X_train)
np.save('./Y_train2', Y_train)
np.save('./X_valid2', X_valid)
np.save('./Y_valid2', Y_valid)

## テストデータの前処理

In [15]:
"""
for i, name in enumerate(img_test_li):
    
    # 画像の読み込み
    img_name = name
    img = cv2.imread(img_name)
        
    # リサイズ
    img = scale_to_width(img, 224)

    # 左右をトリミング
    num_trim = 20
    img = img[:, num_trim:-num_trim]
    
    # リサイズ
    h, w, _ = img.shape
    img = cv2.resize(img, dsize=(224, 224))
    
    # 画像の保存
    img_name_224 = img_name[:12] + "_2_224" + img_name[12:]
    cv2.imwrite(img_name_224, img)
"""
print("保存済みのためコメントアウト")

保存済みのためコメントアウト


In [16]:
# 224サイズ用の画像名リスト作成
img_test_224_li = []
for i, name in enumerate(img_test_li):
    
    img_name = name
    img_name_224 = img_name[:12] + "_2_224" + img_name[12:]
    img_test_224_li.append(img_name_224)

In [17]:
# ndarrayのデータを保管する領域の確保
test_len = len(img_test_224_li)
X_test = np.empty((test_len, 224*224*3), dtype=np.uint8)

for i, name in enumerate(img_test_224_li):

    # 元の画像をndarrayとして読み込んでテストデータに追加
    img_name = name
    img = cv2.imread(img_name)
    
    img = np.asarray(img,dtype=np.uint8)
    img_f = img.flatten()
    X_test[i] = img_f

X_test = X_test.reshape(len(X_test), 224, 224, 3)

In [18]:
# 目的変数の作成
Y_test = [[1, 0]] * len(img_test_n_li) + [[0, 1]] * len(img_test_p_li)
Y_test = np.asarray(Y_test)

In [19]:
# データの保存
np.save('./X_test2', X_test)
np.save('./Y_test2', Y_test)

In [ ]:
# 描画を行う関数
def draw(X):
    fig = plt.figure(figsize=(8, 8))
    pos = 1
    
    for i in range(X.shape[0]):
        ax = fig.add_subplot(4, 4, pos)
        imshow(X[i])
        ax.set_title(f'No.{i+1}', y=0.98)
        ax.axis('off')
        pos += 1
    plt.show()
    
# テスト画像の表示
batch_size = 16
draw(X_test[0:batch_size])